Нестеров Владислав, 569МН група, Варіант 8

Частота f[МГц] = 560
Число елементів антени N = 9

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from scipy.constants import c as light_speed